1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [1]:
import pandas as pd
import numpy as np

#If N>10000 pacco?
N = int(input('Insert starting row index:'))
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
data.iloc[N:1310720, :]

Insert starting row index: 10000


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
10000,1,0,50,3869200267,776,22
10001,1,0,47,3869200267,785,14
10002,1,0,63,3869200267,921,28
10003,1,0,64,3869200267,924,15
10004,1,0,7,3869200267,998,22
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [2]:
N = int(input('Insert starting row index:'))
l = 1

bx_vec = data.loc[:, 'BX_COUNTER']
orbit_vec = data.loc[:, 'ORBIT_CNT']

bx_max = np.zeros(len(data))
for i in range(N,len(data)-1):
    if orbit_vec.iloc[i] != orbit_vec.iloc[i+1]:
        bx_max[l] = bx_vec.iloc[i]
        l = l+1
    
bx_max = [bx_max[i] for i in range(len(bx_max)) if bx_max[i]!=0]
bx_in_orbit = np.max(bx_vec)
print(bx_in_orbit)
#print('The number of BX in a orbit is (for each orbit from labeled by N upwards)', bx_in_orbit)

Insert starting row index: 10000


3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [28]:
import datetime as dt

orbit = data.loc[:,'ORBIT_CNT']*3196*25
bx = data.loc[:,'BX_COUNTER']*25
tdc = data.loc[:,'TDC_MEAS']
first_row = data.loc[0,'ORBIT_CNT']*3196*25+data.loc[0,'BX_COUNTER']*25+data.loc[0,'TDC_MEAS']

#time_s = (orbit + bx + tdc) - first_row
time_s = orbit + bx + tdc

time_s = pd.Series(time_s, name='Time[ns]')
data['Time[ns]'] = time_s
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Time[ns]
0,1,0,123,3869200167,2374,26,309149093402676
1,1,0,124,3869200167,2374,27,309149093402677
2,1,0,63,3869200167,2553,28,309149093407153
3,1,0,64,3869200167,2558,19,309149093407269
4,1,0,64,3869200167,2760,25,309149093412325
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,309149972581964
1310716,1,1,4,3869211171,763,11,309149972581986
1310717,1,0,64,3869211171,764,0,309149972582000
1310718,1,0,139,3869211171,769,0,309149972582125


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [32]:
import datetime as dt

data['Elapsed time']=(pd.to_datetime(data.loc[:,'Time[ns]'])).dt.time
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Time[ns],Elapsed time
0,1,0,123,3869200167,2374,26,309149093402676,13:52:29.093402
1,1,0,124,3869200167,2374,27,309149093402677,13:52:29.093402
2,1,0,63,3869200167,2553,28,309149093407153,13:52:29.093407
3,1,0,64,3869200167,2558,19,309149093407269,13:52:29.093407
4,1,0,64,3869200167,2760,25,309149093412325,13:52:29.093412
...,...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,309149972581964,13:52:29.972581
1310716,1,1,4,3869211171,763,11,309149972581986,13:52:29.972581
1310717,1,0,64,3869211171,764,0,309149972582000,13:52:29.972582
1310718,1,0,139,3869211171,769,0,309149972582125,13:52:29.972582


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [39]:
tdc = pd.Series(data['TDC_CHANNEL'].drop_duplicates())

print(tdc.nlargest(3))

7        139
64511    138
64513    137
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [41]:
N = int(input('Insert starting row index:'))
counter = 0

for i in range(N,len(data)-1):
    if data.loc[i,'ORBIT_CNT'] != data.loc[i+1,'ORBIT_CNT']:
        j = i+1
        if data.loc[j,'ORBIT_CNT'] == data.loc[j+1,'ORBIT_CNT']:
            counter += 1

counter
    

Insert starting row index: 10000


11174

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [80]:

counter = 0

for i in range(len(data)-1):
    if data.loc[i,'TDC_CHANNEL']==139:
        counter += 1

temp = pd.Series(data['TDC_CHANNEL'])
tdc_ch = pd.Series(temp.nlargest(counter))
indx = list(tdc_ch.index)

l = 0
temp_orb = []
for j in range(len(indx)-1):
    temp_orb.append(data.loc[index[j],'ORBIT_CNT'])
    l = l+1

counts = 0
occurrence = {item: temp_orb.count(item) for item in temp_orb}
for k in range(len(indx)-1):
    counts = occurrence.get(str(temp_orb[k]))
    

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [65]:
mask0 = data['FPGA']==0
data_FPGA0 = data[mask0]
FPGA0_s = pd.Series(data_FPGA0['TDC_MEAS'].values, index=data_FPGA0['TDC_CHANNEL'].values)
print(FPGA0_s)

mask1 = data['FPGA']==1
data_FPGA1 = data[mask1]
FPGA0_s = pd.Series(data_FPGA1['TDC_MEAS'].values, index=data_FPGA1['TDC_CHANNEL'].values)
print(FPGA1_s)

123    26
124    27
63     28
64     19
64     25
       ..
60      2
62     14
64      0
139     0
61     18
Length: 922852, dtype: int64
7       4
6      18
139     0
8      14
5       4
       ..
38      8
36     13
39      0
35      4
4      11
Length: 387868, dtype: int64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.